# Advanced data analysis in medicine | Mathematical Statistics

The goal of this notebook is to calculate the cost-efficacy ratio of a dataset (randomized clinical trial on locally advanced lymphoma two treatments were compared with respect to (i) 3 year mortality and (ii) costs of the treatment and other health-care costs incurred by the patients) and its 95% confidence interval.

All mathematical formula's used in this example are elaborated in the pdf 'var_proof.pdf'.

Students: Dilara Tank, Christelle Klein, Nidujaa Rasanajakam

In [1]:
import pandas as pd
import numpy as np

We load the data and remove the unneccessary column patient number. 

In [2]:
df = pd.read_csv('costefficacydata.csv')
df = df.drop('patnr', axis=1)
df

,trt,event,costs
0,1,1,72.611776
1,1,1,35.499437
2,1,1,90.080600
3,1,0,5337.492587
4,1,1,2.360921
...,...,...,...
404,2,0,10.622984
405,2,1,5.854648
406,2,0,7.477292
407,2,1,3.110125


In [3]:
df.isnull().sum()

trt      0
event    0
costs    0
dtype: int64

We check if the number of patients corresponds to what has been given in the assignment (206 in group 1, 203 in group 2).

In [4]:
treatment_1_df = df[df['trt'] == 1] # dataframe includes only data for patients with treatment 1
treatment_2_df = df[df['trt'] == 2] # dataframe includes only data for patients with treatment 2

n1 = len(treatment_1_df)
n2 = len(treatment_2_df)

print(n1)
print(n2)

206
203


In [5]:
def get_survival_rate(df):
    total_survivals = len(df[df['event'] == 1])
    return round(total_survivals / len(df), 4)

def get_mean_costs(df):
    cost_df = df['costs']
    return round(np.mean(cost_df), 4)

In [17]:
# calculate the survival rates and mean costs
p1 = get_survival_rate(treatment_1_df)
p2 = get_survival_rate(treatment_2_df)
m1 = get_mean_costs(treatment_1_df)
m2 = get_mean_costs(treatment_2_df)

# calculate the standard error and confidence interval of the efficacy 
se_p1 = np.sqrt((p1)*(1-p1)/len(treatment_1_df))
se_p2 = np.sqrt((p2)*(1-p2)/len(treatment_2_df))
ci_p1 = (round((p1 - 1.96 * se_p1), 2), round((p1 + 1.96 * se_p1), 2))
ci_p2 = (round((p2 - 1.96 * se_p2), 2), round((p2 + 1.96 * se_p2), 2))

# calculate the standard error and confidence interval of the cost
se_m1 = treatment_1_df['costs'].sem()
se_m2 = treatment_2_df['costs'].sem()
ci_m1 = round((m1 - 1.96 * se_m1)), round((m1 + 1.96 * se_m1))
ci_m2 = round((m2 - 1.96 * se_m2)), round((m2 + 1.96 * se_m2))

print('Mean costs treatment 1 group      (m1): ', m1, ' - 95% CI:', ci_m1)
print('Mean costs treatment 2 group      (m2): ', m2, ' - 95% CI:', ci_m2)
print()
print('Survival rate treatment 1 group   (p1): ', p1, '- 95% CI:', ci_p1)
print('Survival rate treatment 2 group   (p2): ', p2, '- 95% CI:', ci_p2)

Mean costs treatment 1 group      (m1):  995.729  - 95% CI: (582, 1409)
Mean costs treatment 2 group      (m2):  39.8231  - 95% CI: (24, 55)

Survival rate treatment 1 group   (p1):  0.7476 - 95% CI: (0.69, 0.81)
Survival rate treatment 2 group   (p2):  0.6355 - 95% CI: (0.57, 0.7)


We compute the standard errors of the Efficacy E and the Cost C

In [7]:
E = p1 - p2
se_E = round(np.sqrt(se_p1**2 + se_p2**2), 3)
ci_E = (round((E - 1.96 * se_E), 3), round((E + 1.96 * se_E), 3))
print('se(E):', se_E, '95% CI:', ci_E)

se(E): 0.045 95% CI: (0.024, 0.2)


In [8]:
C = m1 - m2
se_C = round(np.sqrt(se_m1**2 + se_m2**2))
ci_C = (round((C - 1.96 * se_C)), round((C + 1.96 * se_C)))
print('se(C):', se_C, '95% CI:', ci_C)

se(C): 211 95% CI: (542, 1369)


C/E ratio = (p2-p1) / (m2-m1) = C / E

In [9]:
CE_ratio = C/E
CE_ratio

8527.26048171275

logCE = log(C/E) = log(C)-log(E)

In [10]:
logCE = (np.log(C)-np.log(E))
logCE

9.051023426148436

Code snippet from [stackabuse](https://stackabuse.com/covariance-and-correlation-in-python/).

In [11]:
def covariance(x, y):
    # Finding the mean of the series x and y
    mean_x = sum(x)/float(len(x))
    mean_y = sum(y)/float(len(y))
    # Subtracting mean from the individual elements
    sub_x = [i - mean_x for i in x]
    sub_y = [i - mean_y for i in y]
    numerator = sum([sub_x[i]*sub_y[i] for i in range(len(sub_x))])
    denominator = len(x)-1
    cov = numerator/denominator
    return cov

In [12]:
n1 = len(treatment_1_df)
n2 = len(treatment_2_df)
cov_CE = covariance(treatment_1_df['event'], treatment_1_df['costs'])/n1 + covariance(treatment_2_df['event'], treatment_2_df['costs'])/n2

We determine the variance of log(CE).

In [13]:
C_part = (se_C**2)/C**2
E_part = (se_E**2)/E**2
cov_part = -2 * cov_CE / (C*E)

var_log_CE = C_part + E_part + cov_part
var_log_CE

0.20428331295860822

se(logCE) = $\sqrt{var[logCE]}$

In [14]:
se_logCE = np.sqrt(var_log_CE)
se_logCE

0.45197711552534187

Now we calculate the CI of the logCE

In [15]:
lower_bound_log = logCE - 1.96 * se_logCE
upper_bound_log = logCE + 1.96 * se_logCE
print('95% CI logCE: ', (round(lower_bound_log, 3), round(upper_bound_log, 3)))

95% CI logCE:  (8.165, 9.937)


Now we calculate the CI of the CE

In [16]:
lower_bound = np.exp(lower_bound_log)
upper_bound = np.exp(upper_bound_log)
print('95% CI CE: ', (round(lower_bound), round(upper_bound)))

95% CI CE:  (3516, 20680)
